#### DACREW 6기 주제  
- [1 .물류 유통량 예측](https://dacon.io/competitions/official/235867/overview/description)  
- [2. 병원 개/폐업 분류 예측](https://dacon.io/competitions/official/9565/overview/description)
- [3. 신용카드 사용자 연체 예측](https://dacon.io/competitions/official/235713/overview/description)
- [4. 주식 종료 가격 예측](https://dacon.io/competitions/official/235857/overview/description)

# [DACON] 신용카드 사용자 연체 예측

##  [대회 설명]
- 신용카드 사용자 데이터를 보고 사용자의 대금 연체 정도를 예측하기

- 평가 기준 : LogLoss

## 1. 필요한 라이브러리 임포트


In [5]:
# 하이퍼파라미터 튜닝에 쓰이는 Automl library
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.6 MB/s eta 0:00:00


In [6]:
# 기본적인 라이브러리
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import itertools

# EDA 시각화 및 전처리.
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 하이퍼파라미터 튜닝에 쓰이는 Automl 기법
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

# 모델링
# from catboost import CatBoostClassifier
from sklearn import svm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# 평가지표
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss

google drive 연결


In [2]:
from google.colab import drive
drive.mount('/content/myDrive')

Mounted at /content/myDrive


In [7]:
path = '/content/myDrive/MyDrive/data/datasets/신용카드 사기 거래 탐지/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

## 2. 데이터 설명  
### [데이터 살펴보기]
> 1. gender: 성별  
> 2. car: 차량 소유 여부  
> 3. realty: 부동산 소유 여부  
> 4. child_num: 자녀 수  
> 5. income_total: 연간 소득  
> 6. income_type: 소득 분류 ['Commercial associate', 'Working', 'State servant', 'Pensioner', 'Student']  
> 7. edu_type: 교육 수준 ['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree']  
> 8. family_type: 결혼 여부 ['Married', 'Civil marriage', 'Separated', 'Single / not married', 'Widow']  
> 9. house_type: 생활 방식 ['Municipal apartment', 'House / apartment', 'With parents', 'Co-op apartment', 'Rented apartment', 'Office apartment']  
> 10. DAYS_BIRTH: 출생일 (데이터 수집 당시 0부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미)  
> 11. DAYS_EMPLOYED: 업무 시작일 (데이터 수집 당시 0부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미. 양수 값은 고용되지 않은 상태를 의미함)  
> 12. FLAG_MOBIL: 핸드폰 소유 여부  
> 13. work_phone: 업무용 전화 소유 여부  
> 14. phone: 가정용 전화 소유 여부  
> 15. email: 이메일 소유 여부  
> 16. occyp_type: 직업 유형  
> 17. family_size: 가족 규모  
> 18. begin_month: 신용카드 발급 월 (데이터 수집 당시 0부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미)  
> 19. credit: 사용자의 신용카드 대금 연체를 기준으로 한 신용도 (낮을 수록 높은 신용의 신용카드 사용자를 의미함)

In [6]:
train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [7]:
train.shape, test.shape

((26457, 20), (10000, 19))

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [9]:
train.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26457.000000,26457.000000,2.645700e+04,26457.000000,26457.000000,26457.0,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000
mean,13228.000000,0.428658,1.873065e+05,-15958.053899,59068.750728,1.0,0.224742,0.294251,0.091280,2.196848,-26.123294,1.519560
std,7637.622372,0.747326,1.018784e+05,4201.589022,137475.427503,0.0,0.417420,0.455714,0.288013,0.916717,16.559550,0.702283
min,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,6614.000000,0.000000,1.215000e+05,-19431.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,1.000000
50%,13228.000000,0.000000,1.575000e+05,-15547.000000,-1539.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,2.000000
75%,19842.000000,1.000000,2.250000e+05,-12446.000000,-407.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,2.000000
max,26456.000000,19.000000,1.575000e+06,-7705.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000,2.000000


In [12]:
# 결측치 확인.
train.isnull().sum()

index               0
gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       8171
family_size         0
begin_month         0
credit              0
dtype: int64

In [1]:
test.info()

NameError: ignored

In [13]:
train.corr()['credit']

index           -0.007841
child_num        0.004081
income_total     0.008555
DAYS_BIRTH      -0.025187
DAYS_EMPLOYED   -0.003798
FLAG_MOBIL            NaN
work_phone      -0.003134
phone            0.003452
email            0.014812
family_size      0.008227
begin_month     -0.147477
credit           1.000000
Name: credit, dtype: float64

In [14]:
# index는 날려도 됨.
train = train.drop(["index"], axis=1)
test = test.drop(["index"], axis=1)